# PhySO-RANS

In [1]:
# External packages
import os
import torch
import numpy as np

# Pyplot
import matplotlib.pyplot as plt

# Internal code import
import physo
from physo.learn import monitoring
from physo.task  import benchmark

In [2]:
# Device
DEVICE = 'cpu'
if torch.cuda.is_available():
    DEVICE = 'cuda'
print(DEVICE)

cpu


In [3]:
# Seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

## Preparing dataset

In [4]:
H = 1.  # Reference height
nu = 5e-06 # Dynamic viscosity

# Index of the 4 independent Reynolds stress components
index = [0, 1, 4, 8]

In [ ]:
# Load the dataset
X = np.load('dataset/q.npy')
y = np.load('dataset/deltaTau.npy')

#### Sending data to device

In [ ]:
# ------ Vectors ------
# Stack of all input variables
X = torch.tensor(X).to(DEVICE)
# Output of symbolic function to guess
y = torch.tensor(y).to(DEVICE)

## Run config

### Library config

In [ ]:
args_make_tokens = {
                # operations
                "op_names"             : ["add", "sub", "mul", "div", "n2",], 
                "use_protected_ops"    : False, 
                # input variables
                "input_var_ids"        : {"q1" : 0         , "q2": 1         , "q3": 2         , "q4": 3         , "q5": 4         , "q6": 5          , "q7": 6           , "q8": 7           , },
                "input_var_units"      : {"q1" : [0, 0, -2], "q2": [0, 2, -2], "q3": [0, 0, 0] , "q4": [0, 2, -3], "q5": [0, 0, 1] , "q6": [0, 1, -2] , "q7": [0, 2, -3]  , "q8": [0, 2, -3]  , }, # [M L T]
                "input_var_complexity" : {"q1" : 1.        , "q2": 1.        , "q3": 1.        , "q4": 1.        , "q5": 1.        , "q6": 1.         , "q7": 1.          , "q8": 1.          , },
                # # constants
                # "constants"            : {"rho" : rho        , },
                # "constants_units"      : {"rho" : [1, -3, 0] , },
                # "constants_complexity" : {"rho" : 1.         , },
                # free constants
                "free_constants"            : {"c"              , },
                "free_constants_init_val"   : {"c" : 1.         , },
                "free_constants_units"      : {"c" : [0, 0, 0]  , },
                "free_constants_complexity" : {"c" : 1.         , },
                    }

library_config = {"args_make_tokens"  : args_make_tokens,
                  "superparent_units" : [0, 2, -2],
                  "superparent_name"  : "y",
                }

                

### Learning config

In [ ]:
reward_config = {
                 "reward_function"     : physo.physym.reward.SquashedNRMSE, # PHYSICALITY
                 "zero_out_unphysical" : True,
                 "zero_out_duplicates" : False,
                 "keep_lowest_complexity_duplicate" : False,
                 "parallel_mode" : False,
                 "n_cpus"        : 12,
                }

In [ ]:
BATCH_SIZE = int(1e4)
MAX_LENGTH = 15
GET_OPTIMIZER = lambda model : torch.optim.Adam(
                                    model.parameters(),                
                                    lr=1e-4, #0.001, #0.0050, #0.0005, #1,  #lr=0.0025
                                                )

In [ ]:
learning_config = {
    # Batch related
    'batch_size'       : BATCH_SIZE,
    'max_time_step'    : MAX_LENGTH,
    'n_epochs'         : 100,
    # Loss related
    'gamma_decay'      : 0.7,
    'entropy_weight'   : 0.005,
    # Reward related
    'risk_factor'      : 0.05,
    'rewards_computer' : physo.physym.reward.make_RewardsComputer (**reward_config),
    # Optimizer
    'get_optimizer'    : GET_OPTIMIZER,
    'observe_units'    : True,
}

### Free constant optimizer config

In [ ]:
free_const_opti_args = {
            'loss'   : "MSE",
            'method' : 'LBFGS',
            'method_args': {
                        'n_steps' : 15,
                        'tol'     : 1e-8,
                        'lbfgs_func_args' : {
                            'max_iter'       : 4,
                            'line_search_fn' : "strong_wolfe",
                                             },
                            },
        }

### Priors config

In [ ]:
priors_config  = [
                #("UniformArityPrior", None),
                # LENGTH RELATED
                ("HardLengthPrior"  , {"min_length": 4, "max_length": MAX_LENGTH, }),
                ("SoftLengthPrior"  , {"length_loc": 8, "scale": 5, }),
                # RELATIONSHIPS RELATED
                ("NoUselessInversePrior"  , None),
                ("PhysicalUnitsPrior", {"prob_eps": np.finfo(np.float32).eps}), # PHYSICALITY
                # ("NestedFunctions", {"functions":["exp", ], "max_nesting" : 1}),
                # ("NestedFunctions", {"functions":["log",], "max_nesting" : 1}),
                # ("NestedTrigonometryPrior", {"max_nesting" : 1}), # 嵌套次数
                # ("OccurrencesPrior", {"targets" : ["nu",], "max" : [2,] }), # 最大出现次数
                 ]

### Cell config

In [ ]:
cell_config = {
    "hidden_size" : 128,
    "n_layers"    : 1,
    "is_lobotomized" : False,
}

### Logger

In [ ]:
run_logger = []
run_visualiser = []

for i in range(len(index)):
    save_path_training_curves = 'RS_curves_' + str(index[i]) + '.png'
    save_path_log             = 'RS_' + str(index[i]) + '.log'

    run_logger.append(monitoring.RunLogger(save_path = save_path_log, 
                                      do_save = False,
                                      ))

    run_visualiser.append(monitoring.RunVisualiser (epoch_refresh_rate = 5,
                                           save_path = save_path_training_curves,
                                           do_show   = True,
                                           do_prints = True,
                                           do_save   = True, 
                                           ))

### Run config

In [ ]:
run_config = []
for i in range(len(index)):
    run_config.append({
        "learning_config"      : learning_config,
        "reward_config"        : reward_config,
        "free_const_opti_args" : free_const_opti_args,
        "library_config"       : library_config,
        "priors_config"        : priors_config,
        "cell_config"          : cell_config,
        "run_logger"           : run_logger[i],
        "run_visualiser"       : run_visualiser[i],
    })

## Learning equations

In [ ]:
for i in range(len(index)):
    benchmark.dummy_epoch(X, y[:, i], run_config[i]) # Dummy epoch for prior tuning
    rewards, candidates = physo.fit (X, y[:, i], run_config[i],
                                    stop_reward = 0.9999, 
                                    stop_after_n_epochs = 5)